In [ ]:
using GZip

using DataFrames

using OnePiece

In [ ]:
gs = "GSE122404"#"GSE107011"

In [ ]:
pa = download(
    "ftp://ftp.ncbi.nlm.nih.gov/geo/series/$(gs[1:end-3])nnn/$gs/soft/$(gs)_family.soft.gz",
)

In [ ]:
function _parse_block(bl)

    ke_va = Dict()

    be = "_table_begin"

    if contains(bl, be)

        ma, ta = split(bl, be)

        li_ = [split(li, '\t') for li in split(ta, '\n', keepempty = false)[1:(end - 1)]]

        da = DataFrame([co => [] for co in popat!(li_, 1)])

        for li in li_

            push!(da, li)

        end

        ke_va["da"] = da

    else

        ma = bl

    end

    for li in split(ma, '\n', keepempty = false)

        if !contains(li, eq)

            println(li)

            continue

        end

        ke, va = split(lstrip(li, '!'), eq, limit = 2)

        id = 2

        while haskey(ke_va, ke)

            ke = "$ke.$id"

            id += 1

        end

        ke_va[ke] = va

    end

    ke_va

end

In [ ]:
eq = " = "

ke_va = nothing

na_pl = Dict()

na_sa = Dict()

for bl in split(read(GZip.open(pa, "r"), String), '^', keepempty = false)

    he, bl = split(bl, '\n', limit = 2)

    ty = split(he, eq)[1]

    if !(ty in ["PLATFORM", "SAMPLE"])

        println(ty)

        continue

    end

    ke_va = _parse_block(bl)

    if ty == "PLATFORM"

        na = ke_va["Platform_geo_accession"]

        na_pl[na] = ke_va

    elseif ty == "SAMPLE"

        na = ke_va["Sample_title"]

        ke = "da"

        if haskey(ke_va, ke)

            println(pop!(ke_va, ke))

        end

        na_sa[na] = ke_va

    end

end

na_pl